In [1]:
import pandas as pd
import numpy as np

In [2]:
# 기업점수 데이터 프레임 호출
df = pd.read_csv('/Users/gimhuidong/Desktop/유비온 공모전, 프로젝트/유비온 2차 프로젝트/코딩폴더/점수화_상용로그 편.csv')

In [3]:
df = df.drop(['회계년', '기업별 표준편차','베타계수'],axis=1)
# '회계년도' 컬럼을 시계열 데이터 타입으로 변경
df['회계년도'] = pd.to_datetime(df['회계년도'])

In [4]:
# 분기 컬럼 추가
def get_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    else:
        return 4
df['분기'] = df['회계년도'].dt.month.apply(get_quarter)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80840 entries, 0 to 80839
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   회사명             80840 non-null  object        
 1   거래소코드           80840 non-null  int64         
 2   회계년도            80840 non-null  datetime64[ns]
 3   순운전자본비율점수       80840 non-null  float64       
 4   부채비율점수          80840 non-null  float64       
 5   재고자산증가율점수       80840 non-null  float64       
 6   총자산증가율점수        80840 non-null  float64       
 7   매입채무회전기간점수      80840 non-null  float64       
 8   매출액순이익률점수       80840 non-null  float64       
 9   자기자본순이익률점수      80840 non-null  float64       
 10  금융비용 대 총비용비율점수  80840 non-null  float64       
 11  총자산사업이익률점수      80840 non-null  float64       
 12  총자산투자효율점수       80840 non-null  float64       
 13  분기별 기업 위험 점수    80840 non-null  float64       
 14  분기              80840 non-null  int64         
dtypes:

In [6]:
df

,회사명,거래소코드,회계년도,순운전자본비율점수,부채비율점수,재고자산증가율점수,총자산증가율점수,매입채무회전기간점수,매출액순이익률점수,자기자본순이익률점수,금융비용 대 총비용비율점수,총자산사업이익률점수,총자산투자효율점수,분기별 기업 위험 점수,분기
0,(주)DB하이텍,990,2011-03-01,8.588183,10.0,1.50515,5.915686,2.334454,4.21442,5.59176,4.771213,4.89279,6.32163,54.135285,1
1,(주)DB하이텍,990,2012-03-01,9.542425,10.0,1.50515,7.605882,2.334454,5.41854,6.29073,4.771213,6.29073,7.22472,60.983844,1
2,(주)DB하이텍,990,2013-03-01,9.542425,10.0,1.80618,6.760784,3.112605,5.41854,6.98970,4.771213,6.29073,7.22472,61.916897,1
3,(주)DB하이텍,990,2014-03-01,9.542425,10.0,1.20412,7.605882,3.890756,5.41854,6.98970,4.771213,6.29073,6.32163,62.034996,1
4,(주)DB하이텍,990,2015-03-01,8.588183,10.0,1.20412,6.760784,3.890756,0.60206,0.69897,4.771213,0.69897,2.70927,39.924326,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80835,성일하이텍(주),365340,2022-12-01,3.816970,4.0,3.01030,0.845098,3.112605,3.61236,4.19382,3.816970,4.19382,4.51545,35.117393,4
80836,솔트웨어(주),328380,2022-12-01,0.954243,5.0,0.30103,8.450980,3.112605,1.20412,0.69897,2.385606,0.69897,2.70927,25.515794,4
80837,오픈엣지테크놀로지(주),394280,2022-12-01,1.908485,5.0,0.00000,6.760784,0.000000,6.02060,6.98970,2.862728,6.98970,9.03090,45.562897,4
80838,와이씨켐(주),112290,2022-12-01,5.725455,6.0,3.01030,2.535294,3.112605,3.61236,4.19382,4.294091,4.19382,5.41854,42.096285,4


- 펀드위험점수 : 해당년도,해당분기 랜덤50종목 분기별 기업위험점수 합
- ex: 2022년 2월 15일 가입이면 2022년 1분기 분기별 기업위험점수 사용
- 가입시점 펀드위험점수 : 전년도 4분기 펀드위험점수 산술평균*가중치 + 전전년도 4분기 펀드위험점수 산술평균*가중치
- 가중치 => 전년도 = 0.7, 전전년도 = 0.3
- 현재시점 펀드위험점수 :
- 현재시점(1,2,4,5,7,8,10,11 월에 포함) : 전전분기 펀드위험점수 사용
- 현재시점(3,6,9,12 월에 포함) : 전분기 펀드위험점수 사용

## 가입시점 펀드 위험지수
-  편입종목 선택가능, 예시는 코스피 상위50기업 포함 

In [7]:
# 에러무시
import warnings 
warnings.filterwarnings('ignore')

In [8]:
# 거래소코드를 기준으로 랜덤으로 50개의 기업을 뽑아오는 코드
import random

selected_companies = df['거래소코드'].unique()
random.shuffle(selected_companies)
unique_companies = selected_companies[:50]
print(unique_companies)

[241840  84110 161890  80470  39010  25620  35620 300080  94940 192820
  85660 363260  78350 217270     20  16670 402030  66410  46890 273060
 321820  73540  84850   6660   1020  92460  42660  86980 161390  24800
 332290  96770  86670  26910  12320 100840  77970  14910 103590   3670
 103140  73190 134790 238200  64520  11210  42500 251270  69460  36000]


In [9]:
# 가입시점 펀드위험점수
# 서로 다른 거래소 코드를 가지는 기업의 리스트 가져오기
unique_companies = df['거래소코드'].unique()[:50] 

# 선택한 기업의 모든 데이터 추출 및 합치기
selected_data = pd.DataFrame()

for company in unique_companies:
    company_data = df[df['거래소코드'] == company]
    selected_data = selected_data.append(company_data)

# 중복 제거하여 결과 출력, # 서로다른 50개 기업 뽑히는거 검증완료
selected_companies = selected_data.drop_duplicates()

# 날짜를 입력받음 (예시: '2022-07-01')
input_date = input("날짜를 입력하세요 (YYYY-MM-DD): ")
input_year = int(input_date[:4])  # 입력한 날짜의 연도

# 입력한 날짜의 전년도와 전전년도 계산
last_year = input_year - 1
previous_year = input_year - 2

# 입력한 날짜의 전년도와 전전년도의 회계년도 계산
last_year_accounting_year = last_year
previous_year_accounting_year = previous_year

# 가입시점 펀드위험점수 계산
selected_companies['회계년도'] = selected_companies['회계년도'].dt.year
filtered_last_year = selected_companies[selected_companies['회계년도'] == last_year_accounting_year]
filtered_previous_year = selected_companies[selected_companies['회계년도'] == previous_year_accounting_year]

weighted_risk_scores = (
    ((filtered_last_year[filtered_last_year['분기'] == 1]['분기별 기업 위험 점수'].sum() +
      filtered_last_year[filtered_last_year['분기'] == 2]['분기별 기업 위험 점수'].sum() +
      filtered_last_year[filtered_last_year['분기'] == 3]['분기별 기업 위험 점수'].sum() +
      filtered_last_year[filtered_last_year['분기'] == 4]['분기별 기업 위험 점수'].sum()) / 4) * 0.7 +
    ((filtered_previous_year[filtered_previous_year['분기'] == 1]['분기별 기업 위험 점수'].sum() +
      filtered_previous_year[filtered_previous_year['분기'] == 2]['분기별 기업 위험 점수'].sum() +
      filtered_previous_year[filtered_previous_year['분기'] == 3]['분기별 기업 위험 점수'].sum() +
      filtered_previous_year[filtered_previous_year['분기'] == 4]['분기별 기업 위험 점수'].sum()) / 4) * 0.3
)

print(f"가입시점 펀드위험점수: {weighted_risk_scores:.4f}")

가입시점 펀드위험점수: 2029.6041


In [10]:
# t-1년도 검증
filtered_last_year

,회사명,거래소코드,회계년도,순운전자본비율점수,부채비율점수,재고자산증가율점수,총자산증가율점수,매입채무회전기간점수,매출액순이익률점수,자기자본순이익률점수,금융비용 대 총비용비율점수,총자산사업이익률점수,총자산투자효율점수,분기별 기업 위험 점수,분기
11,(주)DB하이텍,990,2022,2.862728,3.0,2.70927,0.845098,3.112605,0.60206,0.69897,1.908485,0.69897,0.90309,17.341276,1
23,(주)DB하이텍,990,2022,2.862728,3.0,2.70927,0.845098,7.003361,0.60206,0.69897,2.862728,0.69897,0.90309,22.186274,2
35,(주)DB하이텍,990,2022,1.908485,3.0,2.10721,0.845098,3.112605,0.60206,0.69897,3.816970,0.69897,0.90309,17.693458,3
47,(주)DB하이텍,990,2022,1.908485,3.0,1.80618,0.845098,3.112605,0.60206,1.39794,3.339849,1.39794,0.90309,18.313247,4
59,(주)HJ중공업,97230,2022,7.633940,10.0,0.90309,3.380392,5.447059,4.81648,6.29073,3.816970,5.59176,6.32163,54.202051,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,(주)대한항공,3490,2022,7.633940,9.0,2.10721,3.380392,1.556303,1.80618,1.39794,3.816970,2.79588,2.70927,36.204085,4
2273,(주)대현,16090,2022,2.862728,2.0,1.50515,3.380392,5.447059,1.80618,2.09691,0.954243,1.39794,1.80618,23.256781,1
2285,(주)대현,16090,2022,3.816970,2.0,1.50515,3.380392,7.781513,1.80618,2.79588,0.954243,2.09691,1.80618,27.943417,2
2297,(주)대현,16090,2022,3.816970,2.0,1.80618,3.380392,6.225210,1.80618,2.79588,0.954243,2.09691,3.61236,28.494325,3


In [11]:
# t-2년도 검증
filtered_previous_year

,회사명,거래소코드,회계년도,순운전자본비율점수,부채비율점수,재고자산증가율점수,총자산증가율점수,매입채무회전기간점수,매출액순이익률점수,자기자본순이익률점수,금융비용 대 총비용비율점수,총자산사업이익률점수,총자산투자효율점수,분기별 기업 위험 점수,분기
10,(주)DB하이텍,990,2021,4.771213,4.0,1.20412,4.225490,3.112605,0.60206,0.69897,2.385606,0.69897,0.90309,22.602124,1
22,(주)DB하이텍,990,2021,4.771213,4.0,1.20412,3.380392,7.003361,0.60206,0.69897,1.908485,0.69897,0.90309,25.170661,2
34,(주)DB하이텍,990,2021,3.816970,4.0,1.80618,2.535294,2.334454,0.60206,0.69897,1.908485,0.69897,0.90309,19.304473,3
46,(주)DB하이텍,990,2021,3.816970,4.0,2.40824,1.690196,2.334454,0.60206,0.69897,1.908485,0.69897,0.90309,19.061435,4
58,(주)HJ중공업,97230,2021,8.588183,10.0,0.60206,7.605882,5.447059,5.41854,6.98970,4.294091,6.29073,8.12781,63.364055,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,(주)대한항공,3490,2021,8.588183,10.0,1.80618,4.225490,1.556303,1.20412,0.69897,4.294091,2.09691,1.80618,36.276427,4
2272,(주)대현,16090,2021,1.908485,2.0,0.90309,5.070588,5.447059,2.40824,2.79588,0.954243,2.09691,2.70927,26.293764,1
2284,(주)대현,16090,2021,1.908485,2.0,0.90309,4.225490,7.003361,2.40824,3.49485,0.954243,2.79588,4.51545,30.209089,2
2296,(주)대현,16090,2021,2.862728,3.0,0.90309,5.070588,6.225210,3.01030,4.19382,0.954243,3.49485,5.41854,35.133368,3


## 현재시점 펀드 위험지수
- 편입종목 선택가능, 예시는 코스피 상위50기업 포함 

In [12]:
# 현재시점 펀드 위험지수 
# 서로 다른 거래소 코드를 가지는 기업의 리스트 가져오기
unique_companies = df['거래소코드'].unique()[:50] 

# 선택한 기업의 모든 데이터 추출 및 합치기
selected_data = pd.DataFrame()

for company in unique_companies:
    company_data = df[df['거래소코드'] == company]
    selected_data = selected_data.append(company_data)

# 중복 제거하여 결과 출력, # 서로다른 50개 기업 뽑히는거 검증완료
selected_companies = selected_data.drop_duplicates()

# 입력받은 날짜 정보
input_date = input("날짜를 입력하세요 (YYYY-MM 형식): ")
input_datetime = pd.to_datetime(input_date)

# 분기 계산
input_month = input_datetime.month

if input_month in [1, 2]:
    target_year = input_datetime.year - 1
    target_quarter = 3
elif input_month in [3, 4, 5]:
    target_year = input_datetime.year - 1
    target_quarter = 4
elif input_month in [6, 7, 8]:
    target_year = input_datetime.year
    target_quarter = 1
elif input_month in [9, 10, 11]:
    target_year = input_datetime.year
    target_quarter = 2
elif input_month in [12]:
    target_year = input_datetime.year
    target_quarter = 3

company_risk_score = selected_companies[(selected_companies['분기'] == target_quarter) & (selected_companies['회계년도'].dt.year == target_year)]
current_risk_score = company_risk_score['분기별 기업 위험 점수'].sum()

print(f"현재시점 펀드위험점수: {current_risk_score:.4f}")

현재시점 펀드위험점수: 2205.1525


In [13]:
# 현재 시점 펀드 검증
target_year
target_quarter

4

In [14]:
# 현재 시점 펀드 검증
current_risk_score

2205.152456659863

In [15]:
# 현재 시점 펀드 검증
company_risk_score

,회사명,거래소코드,회계년도,순운전자본비율점수,부채비율점수,재고자산증가율점수,총자산증가율점수,매입채무회전기간점수,매출액순이익률점수,자기자본순이익률점수,금융비용 대 총비용비율점수,총자산사업이익률점수,총자산투자효율점수,분기별 기업 위험 점수,분기
47,(주)DB하이텍,990,2022-12-01,1.908485,3.0,1.80618,0.845098,3.112605,0.60206,1.39794,3.339849,1.39794,0.90309,18.313247,4
95,(주)HJ중공업,97230,2022-12-01,7.633940,10.0,2.40824,3.380392,3.890756,4.21442,5.59176,2.862728,5.59176,6.32163,51.895626,4
143,(주)MH에탄올,23150,2022-12-01,9.542425,10.0,1.20412,5.915686,1.556303,2.40824,3.49485,4.771213,4.19382,3.61236,46.699016,4
191,(주)SH에너지화학,2360,2022-12-01,0.954243,3.0,2.70927,5.070588,1.556303,4.81648,5.59176,2.862728,5.59176,7.22472,39.377851,4
239,(주)SIMPAC,9160,2022-12-01,3.816970,5.0,2.40824,4.225490,1.556303,4.81648,5.59176,4.294091,5.59176,4.51545,41.816544,4
287,(주)SNT에너지,100840,2022-12-01,2.862728,5.0,3.01030,1.690196,5.447059,4.21442,4.89279,4.771213,4.89279,6.32163,43.103125,4
335,(주)SNT홀딩스,36530,2022-12-01,2.862728,4.0,2.40824,3.380392,5.447059,4.81648,5.59176,4.294091,5.59176,2.70927,41.101780,4
383,(주)STX,11810,2022-12-01,8.588183,10.0,0.90309,3.380392,3.890756,4.81648,6.98970,4.294091,6.29073,6.32163,55.475052,4
431,(주)TCC스틸,2710,2022-12-01,7.633940,8.0,2.70927,0.845098,4.668908,1.80618,0.69897,0.477121,0.69897,4.51545,32.053907,4
479,(주)강원랜드,35250,2022-12-01,5.725455,2.0,2.10721,3.380392,1.556303,3.01030,4.89279,0.477121,4.19382,5.41854,32.761931,4


## LBK펀드 위험지수 = 현재시점 펀드 위험지수 / 가입시점 펀드 위험지수

In [16]:
LBK_fund_index = current_risk_score/ weighted_risk_scores

In [17]:
LBK_fund_index

1.0864938883597346